In [1]:
import pandas as pd
import numpy as np
import gretel_client
import os
from gretel_client.synthetics import generate_tabular_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

ModuleNotFoundError: No module named 'gretel_client.synthetics'

In [ ]:


# 1. API 키 설정
config = gretel_client.configure_session(api_key="my_key")



# 1. 데이터 로드 및 전처리 
pd.set_option('display.max_columns', None)
path = '../../../../../archive/raw/HI-Small_Trans.csv'
df = pd.read_csv(path)

# 3. 데이터 전처리
# 2. Timestamp 컬럼을 datetime 객체로 변환 (errors='coerce' 옵션 사용)
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M', errors='coerce')

# 3. 'Timestamp' 컬럼에서 NaT인 값 (변환 실패한 값)을 NaN으로 대체
df['Timestamp'] = df['Timestamp'].replace({pd.NaT: np.nan})

# 4. 결측치 처리 (NaN 값을 삭제 또는 대체)
df = df.dropna(subset=['Timestamp'])

# 5. DataFrame이 비어있는지 확인
if df.empty:
    print("DataFrame is empty after removing rows with missing Timestamp values.")
    # 이후 코드를 실행하지 않도록 처리 (예: return)
    exit()

# 6. datetime 객체를 Unix timestamp로 변환
df['Timestamp'] = df['Timestamp'].astype('int64') // 10**9

# (선택 사항) 필요에 따라 스케일링 또는 정규화를 수행
# 예: MinMaxScaler를 사용하여 0과 1 사이의 값으로 스케일링
scaler = MinMaxScaler()
df['Timestamp'] = scaler.fit_transform(df[['Timestamp']])

# 7. Object 타입 컬럼 원핫 인코딩 (대상 컬럼 줄이기)
# 예: 'Receiving Currency', 'Payment Currency', 'Payment Format' 컬럼만 원핫 인코딩
df = pd.get_dummies(df, columns=['Receiving Currency', 'Payment Currency', 'Payment Format'])

# 8. 'Account', 'Account.1' 컬럼은 라벨 인코딩
label_encoder = LabelEncoder()
df['Account'] = label_encoder.fit_transform(df['Account'])
df['Account.1'] = label_encoder.fit_transform(df['Account.1'])

# 4. 모델 생성 및 학습
try:
    model = generate_tabular_model(
        df.to_csv(index=False),  # DataFrame을 CSV 문자열로 변환
        synthetics_engine="lstm", # 또는 다른 지원되는 엔진
        field_configs={
          "Timestamp": {"data_type": "datetime"} # datetime 타입 명시
        }
    )
    model.train()

    # 5. 합성 데이터 생성
    synthetic_data = model.create_records(num_records=1000)
    synthetic_df = pd.DataFrame.from_records(synthetic_data)
    print(synthetic_df.head())

except Exception as e:
    print(f"Error during Gretel Synthetics processing: {e}")